In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http import models

In [ ]:
%pip install qdrant-client

In [2]:
client = QdrantClient('localhost', port=6333)

In [12]:
client.create_collection(
    collection_name='hello_world_collection',
    vectors_config=models.VectorParams(size=4, distance=models.Distance.COSINE))

True

In [ ]:
models.VectorParams

In [5]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='hello_world_collection'), CollectionDescription(name='multi_vector_collection'), CollectionDescription(name='star_charts')])

In [4]:
client.create_collection(
    collection_name='multi_vector_collection',
    vectors_config = {
        'vector_set_1': models.VectorParams(size=1024, distance=models.Distance.DOT),
        'vector_set_2': models.VectorParams(size=2048, distance=models.Distance.COSINE)
    },
)

True

In [7]:
client.create_collection(
    collection_name='sparse_vector_collection',
    sparse_vectors_config= {
        'vector_set_sparse': models.SparseVectorParams(),
    },
    vectors_config = {}
)

True

In [8]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='hello_world_collection'), CollectionDescription(name='sparse_vector_collection'), CollectionDescription(name='multi_vector_collection'), CollectionDescription(name='star_charts')])

In [9]:
client.delete_collection(collection_name='hello_world_collection')

True

In [13]:
# simple point insert

client.upsert(
    collection_name='hello_world_collection',
    points=[
        models.PointStruct(
            id=1,
            payload={
                'name': 'apple'
            },
            vector=[1.1, 0.1, -2.1, 0.05],
        ),
    ],
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [15]:
# Batch insert
client.upsert(
    collection_name='hello_world_collection',
    points = models.Batch(
        ids=[2,3],
        payloads = [
            {'name': 'orange'},
            {'name': 'peach'},
        ],
        vectors = [
            [3.24, 7.1, 2.8, 1.15],
            [1.1, -0.2, -2.1, 3.21],
        ],
    ),
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [16]:
client.retrieve(
    collection_name='hello_world_collection',
    ids=[1,2,3]
)

[Record(id=1, payload={'name': 'apple'}, vector=None, shard_key=None, order_value=None),
 Record(id=2, payload={'name': 'orange'}, vector=None, shard_key=None, order_value=None),
 Record(id=3, payload={'name': 'peach'}, vector=None, shard_key=None, order_value=None)]

In [17]:
# Update the vectors

client.update_vectors(
    collection_name='hello_world_collection',
    points=[
        models.PointVectors(
            id=2,
            vector=[0.0,1.0, 0.0, 1.0]
        )
    ],
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [22]:
client.retrieve(
    collection_name='hello_world_collection',
    ids=[2],
    with_payload=True,
    with_vectors=True
)

[Record(id=2, payload={'name': 'orange'}, vector={}, shard_key=None, order_value=None)]

In [21]:
# delete the named vectors

client.delete_vectors(
    collection_name='hello_world_collection',
    points=[2],
    vectors=[''],)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [23]:
# Set Payload
client.set_payload(
    collection_name='hello_world_collection',
    payload={
        'property1': 'string',
        'property2': 'string',
    },
    points=[1],
)

UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

In [24]:
client.retrieve(
    collection_name='hello_world_collection',
    ids=[1],
    with_payload=True,
    with_vectors=True
)

[Record(id=1, payload={'name': 'apple', 'property1': 'string', 'property2': 'string'}, vector=[0.4634919, 0.042135626, -0.88484806, 0.021067813], shard_key=None, order_value=None)]

In [26]:
# Overwrite payload

client.overwrite_payload(
    collection_name='hello_world_collection',
    payload={
        'property1':'newstring',
        'property2':'newstring',
    },
    points=[2],
)

UpdateResult(operation_id=5, status=<UpdateStatus.COMPLETED: 'completed'>)

In [27]:
# Clear Payload

client.clear_payload(
    collection_name='hello_world_collection',
    points_selector=models.PointIdsList(
        points=[3],
    ),
)

UpdateResult(operation_id=6, status=<UpdateStatus.COMPLETED: 'completed'>)

In [28]:
client.retrieve(
    collection_name='hello_world_collection',
    ids=[3],
    with_payload=True,
    with_vectors=True
)

[Record(id=3, payload={}, vector=[0.27530906, -0.05005619, -0.52558994, 0.8034018], shard_key=None, order_value=None)]

In [32]:
# Delete payload keys

client.delete_payload(
    collection_name='hello_world_collection',
    keys=['property1', 'property2', 'name'],
    points=[1,2],
)

UpdateResult(operation_id=8, status=<UpdateStatus.COMPLETED: 'completed'>)

In [33]:
client.retrieve(
    collection_name='hello_world_collection',
    ids=[1,2],
    with_payload=True,
    with_vectors=True
)

[Record(id=1, payload={}, vector=[0.4634919, 0.042135626, -0.88484806, 0.021067813], shard_key=None, order_value=None),
 Record(id=2, payload={}, vector={}, shard_key=None, order_value=None)]

In [34]:
# Delete Points

client.delete(
    collection_name='hello_world_collection',
    points_selector=models.PointIdsList(
        points=[3],
    ),
)

UpdateResult(operation_id=9, status=<UpdateStatus.COMPLETED: 'completed'>)

In [35]:
client.retrieve(
    collection_name='hello_world_collection',
    ids=[3],
    with_payload=True,
    with_vectors=True
)

[]